In [21]:
from sklearn.datasets import fetch_openml
mnist=fetch_openml("mnist_784", version=1, as_frame=False)

C:\Users\Ryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [22]:
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=42)


In [23]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
rf_clf = RandomForestClassifier(n_estimators=100,n_jobs=-1)
et_clf = ExtraTreesClassifier(n_estimators=100,n_jobs=-1)
svm_clf = AdaBoostClassifier(LinearSVC(max_iter=100, tol=20),algorithm="SAMME")
knn_clf = KNeighborsClassifier(n_jobs=-1)
mlp_clf = MLPClassifier()

In [24]:
models = [rf_clf, et_clf, svm_clf, knn_clf, mlp_clf]
for model in models:
    print("Training: ", model)
    model.fit(X_train, y_train)
    print("Done")

Training:  RandomForestClassifier(n_jobs=-1)
Done
Training:  ExtraTreesClassifier(n_jobs=-1)
Done
Training:  AdaBoostClassifier(algorithm='SAMME', estimator=LinearSVC(max_iter=100, tol=20))
Done
Training:  KNeighborsClassifier(n_jobs=-1)
Done
Training:  MLPClassifier()
Done


In [26]:
for model in models:
    print(f"{model}: {model.score(X_val,y_val)}")

RandomForestClassifier(n_jobs=-1): 0.9692
ExtraTreesClassifier(n_jobs=-1): 0.9716
AdaBoostClassifier(algorithm='SAMME', estimator=LinearSVC(max_iter=100, tol=20)): 0.9094
KNeighborsClassifier(n_jobs=-1): 0.9702
MLPClassifier(): 0.9626


In [27]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier([
    ("random_forest",rf_clf),
    ("extra_trees",et_clf),
    ("svm_clf",svm_clf),
    ("knn_clf",knn_clf),
    ("mlp_clf",mlp_clf)
])
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest',
                              RandomForestClassifier(n_jobs=-1)),
                             ('extra_trees', ExtraTreesClassifier(n_jobs=-1)),
                             ('svm_clf',
                              AdaBoostClassifier(algorithm='SAMME',
                                                 estimator=LinearSVC(max_iter=100,
                                                                     tol=20))),
                             ('knn_clf', KNeighborsClassifier(n_jobs=-1)),
                             ('mlp_clf', MLPClassifier())])

In [28]:
voting_clf.score(X_val, y_val)

0.9756

In [29]:
del voting_clf.estimators_[2]

In [30]:
voting_clf.score(X_val, y_val)

0.9771

In [31]:
voting_clf.score(X_test, y_test)

0.973

In [32]:
import joblib

joblib.dump(voting_clf,"ensembleClassifier.pk1")

['ensembleClassifier.pk1']

In [35]:
import numpy as np
X_val_predictions = np.empty((len(X_val), len(models)), dtype=np.float32)

for index, model in enumerate(models):
    X_val_predictions[:, index] = model.predict(X_val)

In [36]:
from sklearn.model_selection import cross_val_score
rf_blender = RandomForestClassifier(n_estimators=200,n_jobs=-1)
cross_val_score(rf_blender,X_val_predictions,y_val,scoring="accuracy")

array([0.9775, 0.9765, 0.9725, 0.972 , 0.9775])

In [39]:
mlp_blender=MLPClassifier(max_iter=1000)
cross_val_score(mlp_blender,X_val_predictions,y_val,scoring="accuracy",n_jobs=-1)

array([0.9715, 0.969 , 0.972 , 0.959 , 0.973 ])

In [40]:
knn_blender=KNeighborsClassifier()
cross_val_score(knn_blender,X_val_predictions,y_val,scoring="accuracy",n_jobs=-1)

array([0.9685, 0.9675, 0.9635, 0.9625, 0.9655])

In [43]:
rf_blender.fit(X_val_predictions,y_val)

RandomForestClassifier(n_estimators=200, n_jobs=-1)

In [44]:
X_test_predictions = np.empty((len(X_test), len(models)), dtype=np.float32)

for index, model in enumerate(models):
    X_test_predictions[:, index] = model.predict(X_test)
y_predict = rf_blender.predict(X_test_predictions)

In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.9705

In [47]:
joblib.dump(rf_blender,"rfBlenderClassifier.pk1")

['rfBlenderClassifier.pk1']